<a href="https://colab.research.google.com/github/AMfeta99/Generative_AI/blob/main/Dataset_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dataset Preparation & Augmentation for Image Colorization

This notebook contains the Python script for preparing and augmenting the COCO 2017 dataset for image colorization tasks. The download_dataset.sh script downloads the dataset, followed by dataset_filter.py which refines the dataset by removing images with low color variation. Then, dataset_prepare.py filters removed images and pushes modified datasets to Hugging Face Hub.

In [ ]:
#install requirements
!pip install datasets
!pip install tqdm
!pip install pillow
!pip install openCV

ERROR: Could not find a version that satisfies the requirement openCV (from versions: none)
ERROR: No matching distribution found for openCV


In [ ]:
#Import libraries
from datasets import load_dataset

import requests
from io import BytesIO
from PIL import Image
import numpy as np



Flickr30k https://paperswithcode.com/dataset/flickr30k

MPII Human Pose Dataset http://human-pose.mpi-inf.mpg.de/

coco minitrain https://github.com/giddyyupp/coco-minitrain?tab=readme-ov-file

imageNet-Ctest10k https://paperswithcode.com/dataset/imagenet-ctest10k

#### Comparison Table

Dataset	Size	Focus	Diversity	Best For
- Flickr30k	~31,783	Everyday scenes	Moderate	General image colorization
- MPII Human Pose	~25,000	Human poses and actions	Low	Human-centric colorization
- COCO Minitrain	~5,000	General-purpose scenes	High	General image colorization
- ImageNet-CTest10k	10,000	Colorization (natural images)	High	Image colorization of natural scenes

### Load dataset
COCO minitrain is a subset of the COCO train2017 dataset, and contains 25K images (about 20% of the train2017 set) and around 184K annotations across 80 object categories. We randomly sampled these images from the full set while preserving the following three quantities as much as possible:

- proportion of object instances from each class,
- overall ratios of small, medium and large objects,
- per class ratios of small, medium and large objects.

In [ ]:
!pip install gdown

In [ ]:
# Example to load COCO annotations using pycocotools
from pycocotools.coco import COCO

# Load the COCO annotations file
coco = COCO('/path/to/annotations/instances_train2017_minicoco.json')

# Example: Get a list of all categories
categories = coco.loadCats(coco.getCatIds())
print(categories)

# Load a sample image and its annotations (bounding boxes, segmentation, etc.)
img_id = coco.getImgIds()[0]
img = coco.loadImgs(img_id)[0]
print(f"Image details: {img}")

In [ ]:
from datasets import load_dataset

# Load the COCO Minitrain dataset
dataset = load_dataset("coco_minitrain")

# Check the structure of the dataset
print(dataset)

In [ ]:
# Mount Google Drive (if the data is stored there)
from google.colab import drive
drive.mount('/content/drive')

# Install necessary libraries
!pip install pycocotools datasets gdown

# Download the dataset from Google Drive using gdown or manually upload to Colab
!gdown --id <file_id>

# Unzip the dataset if it's in zip format
!unzip /content/drive/MyDrive/datasets/coco_minitrain_25k.zip -d /content/coco_minitrain

# Load COCO annotations using pycocotools
from pycocotools.coco import COCO
annotation_file = '/content/coco_minitrain/annotations/instances_train2017_minicoco.json'
coco = COCO(annotation_file)

# Function to convert COCO annotations to Hugging Face dataset format
from datasets import Dataset, DatasetDict

def create_coco_hf_dataset(coco):
    data = []
    for img_id in coco.getImgIds():
        img = coco.loadImgs(img_id)[0]
        annotations = coco.loadAnns(coco.getAnnIds(imgIds=img_id))
        data_dict = {
            'image_id': img_id,
            'image': img,
            'annotations': annotations
        }
        data.append(data_dict)

    hf_dataset = Dataset.from_list(data)
    return hf_dataset

# Create Hugging Face dataset
coco_hf_dataset = create_coco_hf_dataset(coco)
coco_hf_dataset = DatasetDict({"train": coco_hf_dataset})

# Show the dataset
print(coco_hf_dataset)